In [4]:
# define the model
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Activation, Add, Lambda
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dropout
from keras import optimizers

img_feature_a = Input(shape=(1024*4,))
img_feature_b = Input(shape=(1024*4,))

h_1 = Dense(256, activation='relu')
bn_1 = BatchNormalization()
do_1 = Dropout(0.5)
s = Dense(1)

def aesthetic_layer(x):
    x = h_1(x)
    x = bn_1(x)
    x = do_1(x)
    return s(x)

score_a = aesthetic_layer(img_feature_a)
score_b = aesthetic_layer(img_feature_b)

negated_score_b = Lambda(lambda x: -x, output_shape=(1,))(score_b)
diff = Add()([score_a, negated_score_b])

output = Activation("sigmoid")(diff)

ranknet = Model(inputs=[img_feature_a, img_feature_b], outputs=output)

#optimizer = optimizers.RMSprop(lr=1e-3, rho=0.9, epsilon=1e-08, decay=0.0)
optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
ranknet.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

ranknet.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 4096)          0                                            
____________________________________________________________________________________________________
input_4 (InputLayer)             (None, 4096)          0                                            
____________________________________________________________________________________________________
dense_3 (Dense)                  (None, 256)           1048832     input_3[0][0]                    
                                                                   input_4[0][0]                    
____________________________________________________________________________________________________
batch_normalization_2 (BatchNorm (None, 256)           1024        dense_3[0][0]           

In [5]:
# define the data
import keras.backend as K
import os
import numpy as np
import lmdb
import random
import cPickle as pickle

train_list = pickle.load(open("./data/train_list.pick", 'rb'))
valid_list = pickle.load(open("./data/valid_list.pick", 'rb'))
env = lmdb.open("./data/mobilenet_feature/")
txn = env.begin()
batch_size = 64 

def get_feature(imgid):
    str_feature = txn.get(imgid)
    return np.fromstring(str_feature, np.float32)

def generate_batch(dataset, batch_size, shuffle=True):
    while True:
        if shuffle:
            random.shuffle(dataset)
            
        for index in range(0, len(dataset), batch_size):
            if index + batch_size > len(dataset):
                current_batch_size = len(dataset) - index
            else:
                current_batch_size = batch_size
            index_array = dataset[index:index+current_batch_size]
            
            batch_x1 = np.zeros((current_batch_size, 1024*4), dtype=K.floatx())
            batch_x2 = np.zeros((current_batch_size, 1024*4), dtype=K.floatx())
            batch_y = np.zeros((current_batch_size, ), dtype=K.floatx())
            for i, (img_a, s_a, img_b, s_b, cmp_ret) in enumerate(index_array):
                x1 = get_feature(img_a)
                x2 = get_feature(img_b)
                batch_x1[i] = x1
                batch_x2[i] = x2
                batch_y[i] = cmp_ret
            yield ([batch_x1, batch_x2], batch_y)

In [ ]:
ranknet.fit_generator(generate_batch(train_list, batch_size), steps_per_epoch=len(train_list) // batch_size, epochs=3,
                       validation_data=generate_batch(valid_list, batch_size), validation_steps=len(valid_list) // batch_size)

Epoch 1/3
1537/6947 [=====>........................] - ETA: 24s - loss: 0.6874 - acc: 0.6060

In [ ]:
ranknet.save_weights("./model/ranknet.h5")